Before this, you need to install PynPoint version 0.8.1 by way of : pip install install pynpoint==0.8.1 and trackpy by
pip install trackpy in your anaconda terminal.

$\textbf{Importing the necessary modules from PynPoint, together with necessary python modules for plotting.}$

In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
from astropy.visualization import astropy_mpl_style
from matplotlib.patches import Circle
from astropy import table
from astropy.utils.data import get_pkg_data_filename
from matplotlib.cm import ScalarMappable


from ImageFunctions import UnsharpMaskModule, CorrectDistortionModule_SPHERE
from BadPixelCleaning import BadPixelMapModule_IRDIS
from pynpoint import Pypeline, FitsReadingModule, FitsWritingModule, StackCubesModule, FlatCalibrationModule, \
    BadPixelSigmaFilterModule, NoddingBackgroundModule, AngleCalculationModule, CombineTagsModule, \
    SortParangModule, StackAndSubsetModule, CropImagesModule, DerotateAndStackModule, ReplaceBadPixelsModule, \
    WaffleCenteringModule, PSFpreparationModule, PcaPsfSubtractionModule, ClassicalADIModule, AttributeWritingModule, \
    DarkCalibrationModule, BadPixelMapModule, AddLinesModule, Hdf5WritingModule, Hdf5ReadingModule, FitCenterModule

#if you get a warning from jupyter here, you might need to downgrade to an older version op ipython by starting up an
#anaconda terminal and running : pip install install ipython==7.10.0

$\textbf{Parameters}$

In [3]:
target = "" #here just put the name of the observed target

epoch = "" #just update these parameters with values applicable for your observations.
exptime = ""
exptime_flat = ""

filt = "B_H"
instrument = "SPHERE"
instrument_det = "IRDIS"

path = ""#this should be the path of the target, in this target folder should be an output folder, an input folder and the
#four other python files containing the necessary functions.


pixscale = 0.01227

$\textbf{Define directories}$

In [ ]:
working_place_in = os.path.join(path)
input_place_in = os.path.join(path,'input\\') #the directory in which all your files are. Within this directory
#the files need to be sorted based on nature: dark, sky, science, center, flat etc.
output_place_in = os.path.join(path,'output\\', "master_flat")#here the master flat
#will be put after its creation, which you can then use again for the data reduction

$\textbf{Configuring directories}$

In [ ]:
# Create Working directory for this target
if not os.path.isdir(working_place_in):
    print("Creating directory: %s" % (working_place_in))
    os.makedirs(working_place_in)

# Create directory for results
if not os.path.isdir(output_place_in):
    print("Creating directory: %s" % (output_place_in))
    os.makedirs(output_place_in)



pipeline = Pypeline(working_place_in=path,
                    input_place_in,
                    output_place_in)

$\textbf{Reading the dark data}$

In [ ]:
dark = FitsReadingModule(name_in='read_dark',
                           input_dir=path + 'input\\DARK',
                           image_tag='dark_arr',
                           overwrite=True,
                           check=False,)

$\textbf{Reading the sky data}$

In [ ]:
sky = FitsReadingModule(name_in="sky_reading",
                                input_dir=path + 'Input\\SKY',
                                image_tag="sky_arr")

$\textbf{Median combine all the sky images}$

In [ ]:
sky_median = DerotateAndStackModule(name_in="median_sky_arr",
                                    image_in_tag="sky_arr",
                                    image_out_tag="sky_arr_median",
                                    derotate=False,
                                    stack='median',
                                    extra_rot=0.)

$\textbf{Reading and making the master flat data}$

In [ ]:
m_flat = FitsReadingModule(name_in='m_flat',
                           input_dir=path + 'Input\\FLAT_LAMP',
                           image_tag='flat_arr1',
                           overwrite=True,
                           check=False,)

In [ ]:
master_flat_1 = DarkCalibrationModule(name_in="master_flat",
                                         image_in_tag="flat_arr1",
                                         dark_in_tag="dark_arr",
                                         image_out_tag="masterflat_arr1")

sky_subtraction_master_flat = DarkCalibrationModule(name_in="sky_subtraction_master_flat",
                                                dark_in_tag="sky_arr_median",
                                                image_in_tag="masterflat_arr1",
                                                image_out_tag="masterflat_arr")

In [ ]:
masterflat_write = FitsWritingModule(name_in='masterflat_write',
                           data_tag='masterflat_arr',
                           file_name='masterflat.fits')

In [ ]:
pipeline.add_module(m_flat)#all the modules are added to the pipeline here, which is ran in the following cell.
pipeline.add_module(dark)
pipeline.add_module(sky)
pipeline.add_module(sky_median)
pipeline.add_module(master_flat_1)
pipeline.add_module(sky_subtraction_master_flat)
pipeline.add_module(masterflat_write)

In [ ]:
pipeline.run()#running this will result in the master flat being created and added to the output folder in a new folder called
#master flat.